In [1]:
# Some standard imports
%matplotlib inline
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import scipy.io as scio
import torch.nn.functional as F
from torch.nn import Module, Parameter, init

In [2]:
class ModulatorTrainingNet(nn.Module):
    def __init__(self, signal_dimension, basis_function_length, samples_per_symbol):
        super(ModulatorTrainingNet, self).__init__()
        self._signal_dimension = signal_dimension
        self._basis_function_length = basis_function_length
        self._samples_per_symbol = samples_per_symbol
        self._kernel_size = [2*self._signal_dimension, 2, self._basis_function_length]

        # Initialize weights
        self._basis_functions = torch.nn.Parameter(torch.randn(self._signal_dimension, 2, self._basis_function_length))
        # Fixed weights for combination in Linear layer
        self._combination_weight = torch.Tensor([[1,0,0,-1],[0,1,1,0]])

    def forward(self, symbol_vetor):
        # Manually split the input into two groups to use shared weights (basis_functions)
        symbol_vetor_tuples = torch.chunk(symbol_vetor,chunks=2,dim=1)
        # print(len(symbol_vetor_tuples))
        symbol_vetor_real = symbol_vetor_tuples[0]
        symbol_vetor_imag = symbol_vetor_tuples[1]
        # print(symbol_vetor_real.shape)
        # print(symbol_vetor_imag.shape)

        # Get components for real and imaginary parts
        signal_components_1 = F.conv_transpose1d(symbol_vetor_real, weight=self._basis_functions, bias=None, stride=self._samples_per_symbol, padding=0, output_padding=0, groups=1, dilation=1)
        signal_components_2 = F.conv_transpose1d(symbol_vetor_imag, weight=self._basis_functions, bias=None, stride=self._samples_per_symbol, padding=0, output_padding=0, groups=1, dilation=1)
        # Stack them to form four-channel output
        signal_components = torch.cat((signal_components_1,signal_components_2), dim=1)
        # Transpose channel <-> length
        signal_components_T = torch.transpose(signal_components, dim0=1, dim1=2)
        # Combine components
        signal_tx = F.linear(input=signal_components_T,weight=self._combination_weight,bias=None)
        # signal_tx = self.signal_comb(signal_components_T)
        
        return signal_tx

In [3]:
# Load Data symbol
# Final symbol tensors will have a shape of (Batch, Channel, Length)
Symbol_file = scio.loadmat('./TrainingWaveform/OFDM/OFDMSymbol_batch.mat')
Symbol = Symbol_file['OFDMSymbol_batch']
print(Symbol.shape)
# Symbol matrix has a shape of (Batch, Channel, Length)
signal_dimension = Symbol.shape[1]
print(signal_dimension)
# Extract real and imaginary parts to form the input mat
Symbol_real = np.real(Symbol)
Symbol_imag = np.imag(Symbol)
Symbol_mat = np.concatenate((Symbol_real, Symbol_imag), axis = 1).astype('float32')

# Add a dimension at 0 for Batch
Symbol_tensor = torch.tensor(Symbol_mat)
print(Symbol_tensor.shape)


# Load Waveform
# Final Waveform tensors will have a shape of (Batch, Length, 2)
Waveform_file = scio.loadmat('./TrainingWaveform/OFDM/OFDMSignal_batch.mat')
Waveform = Waveform_file['OFDMSignal_batch']
print(Waveform.shape)
# Waveform matrix has a shape of (Batch, Length)
# Extract real and imaginary parts to form the input mat
Waveform_real = np.real(Waveform)
Waveform_imag = np.imag(Waveform)
Waveform_mat = np.stack((Waveform_real, Waveform_imag), axis = 2).astype('float32')

Waveform_tensor = torch.tensor(Waveform_mat)

print(Waveform_tensor.shape)

(256, 64, 2)
64
torch.Size([256, 128, 2])
(256, 128)
torch.Size([256, 128, 2])


In [4]:
# Configure basis function
basis_fucntion_file = scio.loadmat('./TrainingWaveform/OFDM/InDFTMat.mat')
basis_fucntions = basis_fucntion_file['InDFTMat']
# Basis function matrix has a shape of (Dimensions, Length) 
basis_fucntion_length = basis_fucntions.shape[1]
print(basis_fucntion_length)
# Extract real and imaginary parts of basis functions
basis_real_tensor = torch.Tensor(np.real(basis_fucntions)).unsqueeze(dim=1)
basis_imag_tensor = torch.Tensor(np.imag(basis_fucntions)).unsqueeze(dim=1)
basis_tensor = torch.concat([basis_real_tensor,basis_imag_tensor],dim=1)
basis_tensor = torch.concat([basis_tensor,basis_tensor],dim=0)
# Configure samples per symbol
samples_per_symbol = 64

64


In [5]:
OFDM_Modulator_Train = ModulatorTrainingNet(signal_dimension=signal_dimension, 
                        basis_function_length=basis_fucntion_length,
                        samples_per_symbol=samples_per_symbol)

In [6]:
LR = 0.05
epochs = 300
list_eps = []
list_loss = []

optimizer = torch.optim.Adam(OFDM_Modulator_Train.parameters(), lr=LR)      # Adam optimizer
loss_function = torch.nn.MSELoss()                           # MSE loss

for eps in range(epochs):
    featureout1 = OFDM_Modulator_Train.forward(Symbol_tensor)
    loss = loss_function(featureout1, Waveform_tensor)
    optimizer.zero_grad()
    loss.backward(retain_graph=True)
    optimizer.step()

    if (eps + 1) % 50 == 0:
        print("Loss: ", loss)
        list_eps.append((eps+1)/10)
        list_loss.append(loss.item().__float__())

Loss:  tensor(0.8461, grad_fn=<MseLossBackward0>)
Loss:  tensor(0.0080, grad_fn=<MseLossBackward0>)
Loss:  tensor(3.9101e-05, grad_fn=<MseLossBackward0>)
Loss:  tensor(3.2813e-07, grad_fn=<MseLossBackward0>)
Loss:  tensor(2.4018e-09, grad_fn=<MseLossBackward0>)
Loss:  tensor(2.3975e-11, grad_fn=<MseLossBackward0>)


In [7]:
#Load test symbols
# Load Data symbol
# Final symbol tensors will have a shape of (Batch, Channel, Length)
Symbol_file = scio.loadmat('./TrainingWaveform/OFDM/OFDMSymbol_batch_test.mat')
Symbol = Symbol_file['OFDMSymbol_batch_test']
print(Symbol.shape)
# Symbol matrix has a shape of (Batch, Channel, Length)
signal_dimension = Symbol.shape[1]
print(signal_dimension)
# Extract real and imaginary parts to form the input mat
Symbol_real = np.real(Symbol)
Symbol_imag = np.imag(Symbol)
Symbol_mat = np.concatenate((Symbol_real, Symbol_imag), axis = 1).astype('float32')

# Add a dimension at 0 for Batch
Symbol_tensor = torch.tensor(Symbol_mat)
print(Symbol_tensor.shape)


# Load Waveform
# Final Waveform tensors will have a shape of (Batch, Length, 2)
Waveform_file = scio.loadmat('./TrainingWaveform/OFDM/OFDMSignal_batch_test.mat')
Waveform = Waveform_file['OFDMSignal_batch_test']
print(Waveform.shape)
# Waveform matrix has a shape of (Batch, Length)
# Extract real and imaginary parts to form the input mat
Waveform_real = np.real(Waveform)
Waveform_imag = np.imag(Waveform)
Waveform_mat = np.stack((Waveform_real, Waveform_imag), axis = 2).astype('float32')

Waveform_tensor = torch.tensor(Waveform_mat)

print(Waveform_tensor.shape)

(128, 64, 2)
64
torch.Size([128, 128, 2])
(128, 128)
torch.Size([128, 128, 2])


In [8]:
MAElossFunction = nn.L1Loss()
testWaveform = OFDM_Modulator_Train(Symbol_tensor)
maeLoss = MAElossFunction(Waveform_tensor, testWaveform)
print(maeLoss)

tensor(1.8463e-06, grad_fn=<MeanBackward0>)


In [9]:
# Save trained weight
TrainedWeight = OFDM_Modulator_Train._basis_functions.detach().numpy()
print(TrainedWeight.shape)
TrainedWeight_OFDM = {"TrainedWeight": TrainedWeight}
scio.savemat("TrainedWeight_OFDM.mat", TrainedWeight_OFDM)

(64, 2, 64)
